In [3]:
import openeo
import geopandas as gpd
import pandas as pd
from openeo_classification.landuse_classification import *
from datetime import date
import ipywidgets as widgets
import datetime
import json

In [4]:
con = openeo.connect("https://openeo-dev.vito.be")

#### Target data 
LUCAS, 2018 (land use cover)

#### Input data
From S2: calculation of 7 indices (NDVI, NDMI, NDGI, ANIR, NDRE1, NDRE2, NDRE5) and keeping 2 bands (B06, B12)
From S1: VV, VH and VV/VH
For all of these, 10 features: p25, p50, p75, sd and 6 t-steps, with flexible range

#### Model
Random Forest, trained using custom hyperparameter, 70/30 split

In [5]:
train_test_split, algorithm, nrtrees, mtry, feature_raster, aoi, strat_layer, include_mixed_pixels, start_date, end_date, nr_targets, nr_spp = getStartingWidgets()

Box(children=(Label(value='Train / test split:'), FloatSlider(value=0.3, max=1.0, step=0.05)))

Dropdown(description='Model:', disabled=True, options=('Random Forest',), value='Random Forest')

Box(children=(Label(value='Hyperparameters RF model:'), IntText(value=250, description='Nr trees:'), IntText(v…

Box(children=(Label(value='S1 / S2 fusion:'), RadioButtons(options=('Feature fusion', 'Decision fusion'), valu…

FileUpload(value={}, accept='.geojson,.shp', description='Upload AOI', layout=Layout(width='20em'))

...

Box(children=(Label(value='Include mixed pixels:'), RadioButtons(options=('Yes', 'No'), value='Yes')))

DatePicker(value=datetime.date(2018, 1, 1), description='Start date')

DatePicker(value=datetime.date(2018, 12, 31), description='End date')

Box(children=(Label(value='Select the amount of target classes:'), IntSlider(value=10, max=37, min=2)))

Box(children=(Label(value='Select the amount of times you want to point sample each reference polygon:'), IntS…

In [6]:
target_classes = getTargetClasses(nr_targets)

SelectMultiple(description='Target class', options=('A00: Artificial land', 'A10: Roofed built-up areas', 'A20…

SelectMultiple(description='Target class', options=('A00: Artificial land', 'A10: Roofed built-up areas', 'A20…

In [7]:
target_classes["target1"].value

('B20: Root crops',
 'B30: Non-permanent industrial crops',
 'B40: Dry pulses, vegetables and flowers',
 'B50: Fodder crops')

In [8]:
mask = gpd.GeoDataFrame.from_features(json.loads(list(aoi.value.values())[0]["content"])).set_crs('epsg:4326')

In [9]:
data = gpd.read_file("https://artifactory.vgt.vito.be/auxdata-public/openeo/LUCAS_2018_Copernicus.gpkg",mask=mask)

if data.empty:
    raise ValueError("Your masked area is located outside of Europe or so small that no training data can be found within it")

In [32]:
# lucas_data = gpd.read_file("lucas/LUCAS_2018_Copernicus_polygons.shp")
# lucas_data["POINT_ID"] = lucas_data["POINT_ID"].astype("int64")
# lucas_attrs = pd.read_csv("lucas/LUCAS_2018_Copernicus_attributes.csv")
# data = lucas_data.merge(lucas_attrs, on='POINT_ID')
# data.to_file('lucas/LUCAS_2018_Copernicus.gpkg', driver='GPKG')

<ipython-input-32-9a2bf50a772b>:3: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  lucas_attrs = pd.read_csv("lucas/LUCAS_2018_Copernicus_attributes.csv")


In [10]:
lucas_points = pd.concat([data]+[data.copy()]*(nr_spp.value-1), ignore_index=True)
lucas_points["geometry"] = lucas_points["geometry"].apply(extract_point_from_polygon)

In [11]:
y = lucas_points[["LC1", "geometry"]]
y["LC1"] = y["LC1"].apply(lambda x: x[:2]+"0")

c:\users\driesseb\.conda\envs\openeo\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [12]:
conv = {value: index for index, value in enumerate(y["LC1"].unique())}
y["LC1"] = y["LC1"].apply(lambda x: conv[x])

In [13]:
if strat_layer is None:
    strat_layer = aoi

In [14]:
## TODO: @Bart load_lc_features spatial_extents toevoegen voor data reductie?
## TODO: @Bart: Loop hier nog over verschillende strata zodra alles werkt, dus:
# for stratum in strat_layer.iterfeatures():
# final_training_data = gpd.clip(data, gpd.GeoDataFrame(stratum))
features, feature_list = load_lc_features(feature_raster, y, start_date.value, end_date.value)

X = features.aggregate_spatial(json.loads(y.to_json()), reducer="mean")

ml_model = X.fit_class_random_forest(target=json.loads(y.to_json()), training=train_test_split.value, num_trees=nrtrees.value, mtry=mtry.value)

model = ml_model.save_ml_model()

Authenticated using refresh token.
Authenticated using refresh token.


In [15]:
training_job = model.create_job()
training_job.start_and_wait()

0:00:00 Job 'd2a0b7e0-2ab9-45d9-997b-7a4d4c75ad78': send 'start'


OpenEoApiError: [400] ProcessParameterRequired: Process 'n/a' parameter 'data' is required. (ref: 03ef2400-738d-4cfb-a2d8-008cc0d7c934)